# Data Collection

In [ ]:
pip install full-fred

In [ ]:
import pandas as pd
import numpy as np
from full_fred.fred import Fred
from datetime import datetime
from time import sleep

In [ ]:
fred = Fred('drive/MyDrive/Machine Learning Project/key.txt') # key.txt contains a valid FRED API key

Download the feature time series from FRED and export them to Google Drive in five separate csv files.  Put each time series is the csv file corresponding to its FRED category.

In [ ]:
def test_duplicate_series(data, new_data):
  data_columns = data.columns
  new_data_columns = new_data.columns
  for column in data_columns:
    if column in new_data_columns and column != 'date':
      return True
  return False

In [ ]:
def find_series(id, categories):
  global data
  if len(categories) == 0: # base case
    try:
      category_series = fred.get_series_in_a_category(id)['seriess']
    except KeyError as error:
      sleep(sleep_time)
      category_series = fred.get_series_in_a_category(id)['seriess']
    new_data = pd.DataFrame()
    for j in range(len(category_series)):
      id = category_series[j]['id']
      title = category_series[j]['title']
      frequency = category_series[j]['frequency_short']
      start_date = datetime.strptime(category_series[j]['observation_start'], '%Y-%m-%d')
      end_date = datetime.strptime(category_series[j]['observation_end'], '%Y-%m-%d')
      if (title.find('(DISCONTINUED)') == -1 and start_date <= min_date and end_date >= max_date and frequency in ('D', 'W', 'M')):
        try:
          series = (fred.get_series_df(series_id = id, frequency = "m"))[['date', 'value']]
        except KeyError as error:
          sleep(sleep_time)
          series = (fred.get_series_df(series_id = id, frequency = "m"))[['date', 'value']]
        for i in range(len(series)):
          series = series.copy()
          series.loc[i, 'date'] = datetime.strptime(series['date'][i], '%Y-%m-%d')
        series = series[series.date >= min_date]
        series = series[series.date <= max_date]
        if (series['value'].isin(['.'])).sum() == 0:
          series.columns = ['date', id]
          if len(new_data.columns) == 0:
            new_data = series
          else:
            new_data = new_data.merge(series, on = 'date')
    return new_data
  else: # recursive step
    for i in range(len(categories)):
      name = categories[i]['name']
      id = categories[i]['id']
      try:
        new_categories = fred.get_child_categories(id)['categories']
      except KeyError as error:
        sleep(sleep_time)
        new_categories = fred.get_child_categories(id)['categories']
      new_data = find_series(id, new_categories)
      if len(data.columns) == 0 and len(new_data.columns) > 0:
        data = new_data
      elif len(data.columns) > 0 and len(new_data.columns) > 0:
        if test_duplicate_series(data, new_data) == False:
          data = data.merge(new_data, on = 'date')
    return data

In [ ]:
def save_series(id, categories, file_path):
  global data
  data = pd.DataFrame()
  df = find_series(id, categories)
  df.to_csv(file_path, index = False)

In [ ]:
data = pd.DataFrame()
sleep_time = 100
min_date = datetime.strptime('1960-01-01', '%Y-%m-%d')
max_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

In [ ]:
# 11m
save_series(32991, fred.get_child_categories(32991)['categories'],
            'drive/MyDrive/Machine Learning Project/data/MoneyBankingAndFinance.csv')

Error Message: Too Many Requests.  Exceeded Rate Limit
Error Message: Too Many Requests.  Exceeded Rate Limit


In [ ]:
# 5m 2s
save_series(10, fred.get_child_categories(10)['categories'][1:],
            'drive/MyDrive/Machine Learning Project/data/PopulationEmploymentAndLaborMarkets.csv')

Error Message: Too Many Requests.  Exceeded Rate Limit


In [ ]:
# 5m 45s
save_series(32992, fred.get_child_categories(32992)['categories'],
            'drive/MyDrive/Machine Learning Project/data/NationalAccounts.csv')

Error Message: Too Many Requests.  Exceeded Rate Limit


In [ ]:
# 2m 26s
save_series(1, fred.get_child_categories(1)['categories'],
            'drive/MyDrive/Machine Learning Project/data/ProductionAndBusinessActivity.csv')

In [ ]:
# 14m
save_series(32455, fred.get_child_categories(32455)['categories'],
            'drive/MyDrive/Machine Learning Project/data/Prices.csv')

Error Message: Too Many Requests.  Exceeded Rate Limit
Error Message: Too Many Requests.  Exceeded Rate Limit


Import the five csv files containg the feature time series from Google Drive and merge them into one data frame.

In [ ]:
df_mbf = pd.read_csv('drive/MyDrive/Machine Learning Project/data/MoneyBankingAndFinance.csv')
print(df_mbf.head())
df_pel = pd.read_csv('drive/MyDrive/Machine Learning Project/data/PopulationEmploymentAndLaborMarkets.csv')
print(df_pel.head())
df_na = pd.read_csv('drive/MyDrive/Machine Learning Project/data/NationalAccounts.csv')
print(df_na.head())
df_pba = pd.read_csv('drive/MyDrive/Machine Learning Project/data/ProductionAndBusinessActivity.csv')
print(df_pba.head())
df_prices = pd.read_csv('drive/MyDrive/Machine Learning Project/data/Prices.csv')
print(df_prices.head())

                  date   AAA   BAA  ...   TOTALSL  TOTALSLAR  TOTALTCU
0  1960-01-01 00:00:00  4.61  5.34  ...  56.01558       0.10     2.923
1  1960-02-01 00:00:00  4.56  5.34  ...  56.36463       7.48     2.850
2  1960-03-01 00:00:00  4.49  5.25  ...  56.86288      10.61     2.810
3  1960-04-01 00:00:00  4.45  5.20  ...  57.71140      17.91     2.963
4  1960-05-01 00:00:00  4.46  5.28  ...  57.95103       4.98     2.867

[5 rows x 109 columns]
                  date  PAYEMS  PAYNSA  ...  CEU9093161101  CEU9093200001  USGOVT
0  1960-01-01 00:00:00   54274   53448  ...         2350.3         2161.7    8307
1  1960-02-01 00:00:00   54513   53422  ...         2380.0         2170.0    8326
2  1960-03-01 00:00:00   54454   53494  ...         2374.7         2182.3    8525
3  1960-04-01 00:00:00   54813   54285  ...         2364.6         2192.4    8534
4  1960-05-01 00:00:00   54475   54349  ...         2351.2         2213.8    8432

[5 rows x 158 columns]
                  date  B039RC1M02

In [ ]:
def merge_dfs(dfs):
  df_all = pd.DataFrame()
  for i in range(len(dfs)):
    if i == 0:
      df_all = dfs[i]
    else:
      df_all = df_all.merge(dfs[i], on = 'date')
  return df_all

In [ ]:
df = merge_dfs([df_mbf, df_pel, df_na, df_pba, df_prices])

for i in range(len(df['date'])):
  df.loc[i, 'date'] = datetime.strptime(df['date'][i].split()[0], '%Y-%m-%d')

Download the two response time series from FRED (unemployment rate and change in unemployment rate) and add them to the data frame containing the feature time series.

In [ ]:
unrate_df = fred.get_series_df('UNRATE', observation_start = '1960-01-01', observation_end = '2020-12-01')[['date', 'value']]

unrate_df = unrate_df.copy()
for i in range(len(unrate_df['date'])):
  unrate_df.loc[i, 'date'] = datetime.strptime(unrate_df['date'][i], '%Y-%m-%d')

for i in range(len(unrate_df['value'])):
  unrate_df.loc[i, 'value'] = float(unrate_df['value'][i])
unrate_df['value'] = unrate_df['value'].astype(np.float)

unrate_df.columns = ['date', 'UNRATE']

In [ ]:
unrate_chg_df = fred.get_series_df('UNRATE', units = 'chg', observation_start = '1960-01-01', observation_end = '2020-12-01')[['date', 'value']]

unrate_chg_df = unrate_chg_df.copy()
for i in range(len(unrate_chg_df['date'])):
  unrate_chg_df.loc[i, 'date'] = datetime.strptime(unrate_chg_df['date'][i], '%Y-%m-%d')

for i in range(len(unrate_chg_df['value'])):
  unrate_chg_df.loc[i, 'value'] = float(unrate_chg_df['value'][i])
unrate_chg_df['value'] = (unrate_chg_df['value'] > 0).astype(np.int)

unrate_chg_df.columns = ['date', 'UNRATECHG']

In [ ]:
df = df.merge(unrate_df, on = 'date')
df = df.merge(unrate_chg_df, on = 'date')
print(df.head())

                  date   AAA   BAA  ...  PCU22112222112243  UNRATE  UNRATECHG
0  1960-01-01 00:00:00  4.61  5.34  ...               17.6     5.2          0
1  1960-02-01 00:00:00  4.56  5.34  ...               17.8     4.8          0
2  1960-03-01 00:00:00  4.49  5.25  ...               17.8     5.4          1
3  1960-04-01 00:00:00  4.45  5.20  ...               17.8     5.2          0
4  1960-05-01 00:00:00  4.46  5.28  ...               17.8     5.1          0

[5 rows x 803 columns]


Export the full data frame to Google Drive.

In [ ]:
df.to_csv('drive/MyDrive/Machine Learning Project/data/AllData.csv', index = False)